# ANM-LD: Predicting Conformational Transition Pathways with Intrinsic Dynamics

> Note: The Colab notebook is still in beta. If you encounter any errors, you can send them to ahmedselimuzum@gmail.com or open a github issue and use the local version with the AMBER backend.

<p align="center"><img src="https://github.com/ahmedselim2017/anmld_python/blob/662a4ae30814c23bb206c57bbd0b807dcbbdc660/anmld.png?raw=true"/></p>
<p align="center"><img src="https://github.com/ahmedselim2017/anmld_python/blob/main/figures/anmld_aa1r.gif?raw=true" height="250"/><img src="https://github.com/ahmedselim2017/anmld_python/blob/main/figures/anmld_adk.gif?raw=true" height="250"/><img src="https://github.com/ahmedselim2017/anmld_python/blob/main/figures/anmld_serca.gif?raw=true" height="250"/></p>



In [ ]:
#@title Install ANM-LD & Initialize run

# openmm-cuda12 is not supported on colab because it uses unsupported PTX version
# therefore, the OpenCL backend is used to run on GPU
!pip install -q --no-warn-conflicts "anmld-python[notebook] @git+https://github.com/ahmedselim2017/anmld_python"

from google.colab import files
from pathlib import Path
from loguru import logger

jobname = "anmld_testrun" #@param {type:"string"}

in_path = Path(f"{jobname}/in")
out_path = Path(f"{jobname}/out")

in_path.mkdir(exist_ok=True, parents=True)

settings_path = in_path / "settings.toml"

#@markdown ---

download_structures_from_PDB = True #@param {type: "boolean"}
initial_pdb_id = "1ake" #@param {type: "string"}
target_pdb_id = "4ake" #@param {type: "string"}
#@markdown - Download the asymmetric unit from PDB and save to {jobname}/in/{pdb_id}.pdb


upload_structures = False #@param {type: "boolean"}
initial_filename = "1ake.pdb" #@param {type: "string"}
target_filename = "4ake.pdb" #@param {type: "string"}
#@markdown - A filepicker option will appear as an output for the current cell for loading structure files. The uploaded files will be saved to {jobname}/in/{filename}



if upload_structures and download_structures_from_PDB:
    raise ValueError("The upload_structures and download_structures_from_PDB options are mutually exclusive")
elif download_structures_from_PDB:
    import biotite.database.rcsb as rcsb

    init_path = Path(rcsb.fetch(initial_pdb_id, "pdb", in_path))
    logger.info(f"Downloaded {init_path}")
    target_path = Path(rcsb.fetch(target_pdb_id, "pdb", in_path))
    logger.info(f"Downloaded {target_path}")
elif upload_structures:
    init_path = in_path / initial_filename
    target_path = in_path / target_filename

    if not (init_path).is_file():
        logger.info("Uploading the initial structure")
        files.upload_file(str(init_path))
    if not (target_path).is_file():
        logger.info("Uploading the target structure")
        files.upload_file(str(target_path))
else:
    raise ValueError("Please select structure input method as upload_structures or download_structures_from_PDB")

#@markdown ---


input_chains = "A" #@param {type: "string"}
target_chains = "A" #@param {type: "string"}

#@markdown - Select multiple chains by entering a comma-separated list (e.g., "A,B,C").

try:
    input_chains = [chain.strip() for chain in input_chains.split(",")]
    target_chains = [chain.strip() for chain in target_chains.split(",")]
except ValueError as e:
    logger.error("Couldn't parse the given chain names")
    raise e


In [ ]:
#@title Run!
from anmld_python.settings import AppSettings, ANMLDSettings,  OpenMMSettings, PathSettings

if out_path.exists():
    n = 1
    while Path(f"{jobname}/out_{n}").exists(): n += 1
    out_path = Path(f"{jobname}/out_{n}")
    logger.warning(f"Changing it to {out_path}.")


loglevel = 'INFO' # @param ["INFO", "DEBUG"]


#@markdown ##ANMLD Settings
N_steps = 100 #@param {type: "integer"}
ANM_cutoff = 8.0 #@param {type: "number"}
ANM_gamma = 1.0 #@param {type: "number"}
deformation_factor = 0.6 #@param {type: "number"}
max_mode = 30 #@param {type: "integer"}
early_stopping_aa_rmsd = 2.0 #@param {type: "number"}
early_stopping_ca_rmsd = 1.0 #@param {type: "number"}
#@markdown - Stops the run when the given all-atom or C-alpha RMSD is reached. To disable, set to 0.


#@markdown ##OpenMM Settings
min_step = 500 #@param {type: "integer"}
ld_step = 100 #@param {type: "integer"}
ld_temp = 310.0 #@param {type: "number"}

save_min = False #@param {type: "boolean"}
#@markdown - Save structures directly after the minimization before the LD
save_ld = False #@param {type: "boolean"}
#@markdown - Save structures directly after the LD before aligning to the target

app_settings = AppSettings(
    logging_level=loglevel,
    mode_selection="ORIGINAL",
    LD_method="OpenMM",
    PATHS=PathSettings(
        out_dir=out_path
    ),
    ANMLD=ANMLDSettings(
        n_steps=N_steps,
        rcut_ANM=ANM_cutoff,
        gamma_ANM=ANM_gamma,
        DF=deformation_factor,
        max_mode=max_mode,
        early_stopping_aa_rmsd=early_stopping_aa_rmsd,
        early_stopping_ca_rmsd=early_stopping_ca_rmsd
    ),
    OpenMM=OpenMMSettings(
        min_step=min_step,
        ld_step=ld_step,
        ld_temp=ld_temp,
        platform="OpenCL",
        save_min=save_min,
        save_ld=save_ld
    ),
)

import toml
with open(settings_path, "w") as f:
    settings_dump = app_settings.model_dump(mode="toml",by_alias=True,exclude_none=True)
    toml.dump(settings_dump, f)
logger.info(f"Saved configration to {settings_path}")

#@title Run!
from anmld_python import main as anmld
from tqdm.notebook import tqdm

if not settings_path.is_file():
    logger.error("Please first run the settings cell above")
elif out_path.is_dir():
    logger.error("The output directory {out_path} already exists. Please create a new output directory by rerunning the 'Initialize run' cell")
else:
    anmld.main(settings_path = settings_path,
            path_abs_structure_init=init_path.absolute(),
            path_abs_structure_target=target_path.absolute(),
            chain_init=input_chains,
            chain_target=target_chains
    )


In [ ]:
#@title Visualize the ANM-LD trajectory
import py3Dmol
pdbs = sorted(out_path.rglob("STEP_*_ANMLD.pdb"))
# Concatenate all PDB file contents into a single string.
multi_model_pdb_string = ""
for i, pdb_file in enumerate(pdbs):
    with open(pdb_file, 'r') as f:
        multi_model_pdb_string += f"MODEL        {i+1}\n"
        multi_model_pdb_string += f.read()
        multi_model_pdb_string += "ENDMDL\n"

# Create a py3Dmol viewer instance.
view = py3Dmol.view(width=600, height=400)

# CORRECTED LINE: Use addModel (singular) for trajectories.
view.addModelsAsFrames(multi_model_pdb_string, 'pdb')

# Set the visualization style.
view.setStyle({'cartoon': {'color': 'spectrum'}})

# Center the camera on the molecule.
view.zoomTo()

# Enable the animation controls.
view.animate({'loop': 'backAndForth'})

# Display the interactive viewer.
view.show()


In [ ]:
#@title Download results
#@markdown - Downloading automatically may not work with adblockers but the packaged zip `{jobname}_results.zip` can be downloaded from the left file explorer sidepane
#@markdown - The final predicted structures are saved as `STEP_[STEP_NUMBER]_ANMLD.pdb`. The intermediate structures, which have been deformed by ANM but have not yet undergone LD, are saved as `STEP_[STEP_NUMBER]_ANM.pdb`.
#@markdown - You can find more information about the run in the `run_info.csv` file.
zipname = f"{jobname}_results.zip"
import os
os.system(f"zip -r {zipname} {out_path}")

files.download(zipname)
logger.info(f"Downloaded {zipname}")


## Citing ANM-LD
If you use ANM-LD, please cite:

```bibtex
@article{ersoy_computational_2023,
	title = {Computational analysis of long-range allosteric communications in {CFTR}},
	volume = {12},
	issn = {2050-084X},
	url = {https://elifesciences.org/articles/88659},
	doi = {10.7554/eLife.88659.3},
	abstract = {Malfunction of the {CFTR} protein results in cystic fibrosis, one of the most common hereditary diseases. {CFTR} functions as an anion channel, the gating of which is controlled by long-range allosteric communications. Allostery also has direct bearings on {CF} treatment: the most effective {CFTR} drugs modulate its activity allosterically. Herein, we integrated Gaussian network model, transfer entropy, and anisotropic normal mode-Langevin dynamics and investigated the allosteric communications network of {CFTR}. The results are in remarkable agreement with experimental observations and mutational analysis and provide extensive novel insight. We identified residues that serve as pivotal allosteric sources and transducers, many of which correspond to disease-causing mutations. We find that in the {ATP}-free form, dynamic fluctuations of the residues that comprise the {ATP}-binding sites facilitate the initial binding of the nucleotide. Subsequent binding of {ATP} then brings to the fore and focuses on dynamic fluctuations that were present in a latent and diffuse form in the absence of {ATP}. We demonstrate that drugs that potentiate {CFTR}’s conductance do so not by directly acting on the gating residues, but rather by mimicking the allosteric signal sent by the {ATP}-binding sites. We have also uncovered a previously undiscovered allosteric ‘hotspot’ located proximal to the docking site of the phosphorylated regulatory (R) domain, thereby establishing a molecular foundation for its phosphorylation-dependent excitatory role. This study unveils the molecular underpinnings of allosteric connectivity within {CFTR} and highlights a novel allosteric ‘hotspot’ that could serve as a promising target for the development of novel therapeutic interventions.},
	pages = {RP88659},
	journaltitle = {{eLife}},
	author = {Ersoy, Ayca and Altintel, Bengi and Livnat Levanon, Nurit and Ben-Tal, Nir and Haliloglu, Turkan and Lewinson, Oded},
	urldate = {2025-09-23},
	date = {2023-12-18},
	langid = {english},
}

@article{ACAR2020651,
title = {Distinct Allosteric Networks Underlie Mechanistic Speciation of ABC Transporters},
journal = {Structure},
volume = {28},
number = {6},
pages = {651-663.e5},
year = {2020},
issn = {0969-2126},
doi = {https://doi.org/10.1016/j.str.2020.03.014},
url = {https://www.sciencedirect.com/science/article/pii/S0969212620300964},
author = {Burçin Acar and Jessica Rose and Burcu {Aykac Fas} and Nir Ben-Tal and Oded Lewinson and Turkan Haliloglu},
keywords = {ABC transporter, membrane protein, molecular dynamics, ANM, ANM-LD, allostery, ATP hydrolysis, transport},
abstract = {Summary
ABC transporters couple the energy of ATP hydrolysis to the transmembrane transport of biomolecules. Here, we investigated the allosteric networks of three representative ABC transporters using a hybrid molecular simulations approach validated by experiments. Each of the three transporters uses a different allosteric network: in the constitutive B12 importer BtuCD, ATP binding is the main driver of allostery and docking/undocking of the substrate-binding protein (SBP) is the driven event. The allosteric signal originates at the cytoplasmic side of the membrane before propagating to the extracellular side. In the substrate-controlled maltose transporter, the SBP is the main driver of allostery, ATP binding is the driven event, and the allosteric signal propagates from the extracellular to the cytoplasmic side of the membrane. In the lipid flippase PglK, a cyclic crosstalk between ATP and substrate binding underlies allostery. These results demonstrate speciation of biological functions may arise from variations in allosteric connectivity.}
}
```

## Further Reading

```bibtex
@article{yang_single-molecule_2018,
	title = {Single-molecule probing of the conformational homogeneity of the {ABC} transporter {BtuCD}},
	volume = {14},
	issn = {1552-4450, 1552-4469},
	url = {https://www.nature.com/articles/s41589-018-0088-2},
	doi = {10.1038/s41589-018-0088-2},
	pages = {715--722},
	number = {7},
	journaltitle = {Nature Chemical Biology},
	shortjournal = {Nat Chem Biol},
	author = {Yang, Min and Livnat Levanon, Nurit and Acar, Burçin and Aykac Fas, Burcu and Masrati, Gal and Rose, Jessica and Ben-Tal, Nir and Haliloglu, Turkan and Zhao, Yongfang and Lewinson, Oded},
	urldate = {2025-09-23},
	date = {2018-07},
	langid = {english},
}
```